warnings: 경고 메시지를 무시하고 싶을 때 사용됩니다.

numpy: 행렬 등의 수치 연산을 빠르게 수행하기 위해 사용되는 선형대수 라이브러리입니다.
pandas: 데이터를 쉽고 유연하게 처리하기 위한 라이브러리입니다.
tqdm: 모델의 학습 과정을 시각화하기 위해 사용됩니다.
product: 여러 iterable의 Cartesian product를 생성합니다.
sklearn.metrics: 모델 성능 평가를 위한 metric을 scikit-learn 라이브러리에서 가져옵니다.
- f1_score: 정밀도와 재현율의 조화평균

matplotlib: 그래프 도시를 위한 도구를 제공하는 라이브러리입니다.



분석을 위해 미리 정의해 둔 파이썬 파일들을 불러옵니다.
- module 디렉터리 내부에 존재하는 여러 파이썬 파일들을 가져옵니다.

1. argument: 파일을 실행할 때, 추가적으로 argument를 받을 수 있도록 하는 함수를 포함합니다.
    1. get_parser: 추가적인 argument로 어떤 변수들을 받을 지 설정합니다.(자세한 내용은 argument.py에서 확인할 수 있음)
        - 하지만, 본 파이썬 파일에서는 가능한 모든 경우에 대해서 실행되기 때문에, 커맨드 라인에서 argument를 받지 않습니다.
        - 따라서 사용되지 않습니다.

2. read_data: 데이터를 불러오는 함수를 포함하고 있는 파이썬 파일입니다.
    1. load_data: 데이터를 불러오는 함수입니다.
    2. multiclass2binary: catetory의 종류를 3개에서 2개로 줄여주는 함수입니다.

3. get_model: 모델을 불러오는 함수를 포함하고 있는 파이썬 파일입니다.
    1. load_model: 모델을 불러오는 함수입니다.

4. common: 분석을 위해 필요한 미리 정의해 둔 여러 잡다한 함수들을 포함하고 있는 파이썬 파일입니다.
    1. data_split: 데이터를 train, validation, test 데이터로 분리하는 함수입니다.
    2. load_val_result: JSON 형식의 검증 결과를 불러오는 함수입니다.
    3. print_best_param: 교차 검증을 수행한 결과를 출력하는 함수입니다.



**plt.rcParams['figure.dpi'] = 300**

그래프의 크기를 조절하기 위한 코드입니다. 


In [ ]:
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product

from sklearn.metrics import f1_score

from module.argument import get_parser
from module.read_data import (
    load_data,
    multiclass2binary
)
from module.get_model import (
    load_model
)
from module.common import (
    data_split, 
    load_val_result,
    print_best_param
)

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300


warnings.filterwarnings('ignore')

main은 머신러닝 실험을 위한 전체 프로세스를 자동화하기 위한 함수입니다.


아래와 같은 과정으로 동작합니다.
1. product 함수를 이용하여 실험에 필요한 조합을 생성해, comb에 저장합니다.
    1. tg_nums: 실험에 사용할 데이터프레임 번호
    2. inhale_types: 실험에 사용할 흡입 방식
    3. model_names: 실험에 사용할 모델
    4. thresholds: 실험에 사용할 임계값
    - 나중에 실험에 사용할 조합만을 가져와 사용하게 됩니다.

2. f1_dict에 실험에 사용할 조합을 키로 하여, 빈 딕셔너리를 생성합니다.
    - 예를 들어 아래와 같은 딕셔너리가 생성됩니다.
    
    f1_dict = {
        403: {
            'vapour': {
                'logistic': {
                    0.1: {},
                    0.2: {},
                    ...
                },
                'lda': {
                    0.1: {},
                    0.2: {},
                    ...
                },
                ...
            },
            'aerosol': {
                'logistic': {
                    0.1: {},
                    0.2: {},
                    ...
                },
                'lda': {
                    0.1: {},
                    0.2: {},
                    ...
                },
                ...
            },
        },...
    }

    - 마크다운 실행 이전의 형태로 보면, 구조를 보다 쉽게 볼 수 있습니다.
    - 위와 같은 중첩 딕셔너리를 만드는 이유는 각 조합에 대한 f1 점수를 저장할 공간을 만들기 위함입니다.

3. tqdm을 이용하여 실험에 사용할 조합을 하나씩 가져와서, 아래의 과정을 반복합니다.
    1. 조합에 해당되는 데이터를 불러옵니다.
    2. 불러온 데이터를 이용하여, 데이터를 분할합니다.(훈련 데이터와 테스트 데이터로 분리합니다.)
    3. 훈련 데이터를 이용하여, 모델을 학습합니다.
    4. 학습된 모델을 이용하여, 테스트 데이터에 대한 예측값을 구합니다.
    5. 구한 예측값을 이용하여, f1 score를 계산합니다.
    6. 계산된 f1 score를 f1_dict에 저장합니다.

4. f1_dict를 반환합니다.


In [ ]:
def main():
    tg_nums = [403, 412, 413]
    inhale_types = ['vapour', 'aerosol']
    model_names = ['logistic', 'lda', 'qda', 'plsda', 'dt', 'rf', 'gbt', 'xgb', 'lgb', 'mlp']
    thresholds = np.round(np.arange(0.1, 1, 0.1), 1).tolist()
    metric = 'f1'
    
    comb = list(product(tg_nums, inhale_types, model_names, thresholds))
    # data_comb = list(product(tg_nums, inhale_types))
    
    f1_dict = {x: {y: {z: {}.fromkeys(thresholds) for z in model_names} for y in inhale_types} for x in tg_nums}
    
    for c in tqdm(comb):
        tg_num, inhale_type, model_name, thres = c
        
        x, y = load_data(path = 'data', tg_num = tg_num, inhale_type = inhale_type)
        y = multiclass2binary(y, tg_num)
    
        result = load_val_result('', tg_num, inhale_type, model_name)
        best_param = print_best_param(val_result = result, metric = metric)
    
        # test reulst
        f1 = []

        for seed in range(10):
            x_train, x_test, y_train, y_test = data_split(x, y, seed)
            
            model = load_model(model = model_name, seed = seed, param = best_param)
            model.fit(x_train, y_train)
            if model_name == 'plsda':
                pred_score = model.predict(x_test)
            else:
                pred_score = model.predict_proba(x_test)[:, 1]
            pred = np.where(pred_score < thres, 0, 1).reshape(-1)
            
            f1.append(f1_score(y_test, pred))
        
            f1_dict[tg_num][inhale_type][model_name][thres] = {'mean': np.mean(f1), 'std': np.std(f1)}
    
    return f1_dict

main 함수를 호출해 모든 조합에 대한 실험을 수행하고 이를 f1_dict에 저장합니다.
아래에서 그래프를 도시할 때, legend를 표시하기 위해 모델들의 이름을 순서대로 리스트, model_names에 저장합니다.

In [ ]:
f1_dict = main()
model_names = ['logistic', 'lda', 'qda', 'plsda', 'dt', 'rf', 'gbt', 'xgb', 'lgb', 'mlp']

tg_num이 403, inhale_type이 vapour인 경우에 해당되는 모델들의 정보만 vapor403에 저장합니다.
모델의 이름을 키로 하고, 각 모델의 f1-score를 값으로 하는 형식으로 데이터프레임을 재정의합니다.
- apply(pd.Series)를 통해 사전 형태의 값을 별도의 열로 확장합니다.

총 2개의 그래프를 그립니다.
1. 모든 모델에 대한 각각의 F1-score의 평균을 하나의 figure에 그립니다.
    - 총 10개의 모델에 대한 F1-score의 평균을 그립니다.
    - 그래프의 이름은 'Acute Vapor F1 score'로 설정합니다.

2. Logistic Regression에 대한 F1-score를 하나의 figure에 그립니다.
    - errorbar은 f1-score 평균과 표준편차를 오차 막대 그래프로 표시하기 위해 사용합니다.
        - f1-score의 변동성을 확인하기 위함입니다.

    - 그래프의 이름은 'Acute Vapor Logistic Regression F1 score'로 설정합니다.

In [ ]:
#
vapor403 = pd.DataFrame(f1_dict[403]['vapour'])
vapor403 = pd.concat([vapor403[x].apply(pd.Series) for x in list(vapor403)], 1, keys=list(vapor403))

plt.figure(figsize = (10, 5))
for m in model_names:
    plt.plot(vapor403.index, vapor403[m]['mean'], '-o', markersize='3')
    # plt.errorbar(vapor403.index, vapor403[m]['mean'], yerr = vapor403[m]['std'], capsize = 4, fmt = '-o', markersize='3')
plt.ylim((0, 1))
plt.legend(model_names, ncol = 2)
plt.title('Actue Vapor F1 score')
plt.show()
plt.close()

m = 'logistic'
plt.figure(figsize = (10, 5))
plt.errorbar(vapor403.index, vapor403[m]['mean'], yerr = vapor403[m]['std'], capsize = 4, fmt = '-o', markersize='3')
plt.ylim((0, 1))
plt.title('Actue Vapor Logistic Regression F1 score')
plt.show()
plt.close()

tg_num이 403, inhale_type이 aerosol인 경우에 해당되는 모델들의 정보만 추출하여 aerosol403에 저장합니다.
모델의 이름을 키로 하고, 각 모델의 f1-score를 값으로 하는 형식으로 데이터프레임을 재정의합니다.
- apply(pd.Series)를 통해 사전 형태의 값을 별도의 열로 확장합니다.

1개의 그래프를 그립니다.
1. 모든 모델에 대한 각각의 F1-score의 평균을 하나의 figure에 그립니다.
    - 그래프의 이름은 'Acute Aerosol F1 score'로 설정합니다.




412_vapour, 412_aerosol, 413_vapour, 413_aerosol에 대해서도 동일한 과정을 거쳐 그래프를 도시합니다.

In [ ]:
#
aerosol403 = pd.DataFrame(f1_dict[403]['aerosol'])
aerosol403 = pd.concat([aerosol403[x].apply(pd.Series) for x in list(aerosol403)], 1, keys = list(aerosol403))

plt.figure(figsize = (10, 5))
for m in model_names:
    plt.plot(aerosol403.index, aerosol403[m]['mean'], '-o', markersize='3')
    # plt.errorbar(vapor403.index, vapor403[m]['mean'], yerr = vapor403[m]['std'], capsize = 4, fmt = '-o', markersize='3')
plt.ylim((0, 1))
plt.legend(model_names, ncol = 2)
plt.title('Actue Aerosol F1 score')
plt.show()
plt.close()

In [ ]:
#
vapor412 = pd.DataFrame(f1_dict[412]['vapour'])
vapor412 = pd.concat([vapor412[x].apply(pd.Series) for x in list(vapor412)], 1, keys = list(vapor412))

plt.figure(figsize = (10, 5))
for m in model_names:
    plt.plot(vapor412.index, vapor412[m]['mean'], '-o', markersize='3')
plt.ylim((0, 1))
plt.legend(model_names, ncol = 2)
plt.title('Sub-Actue Vapor F1 score')
plt.show()
plt.close()

In [ ]:
#
aerosol412 = pd.DataFrame(f1_dict[412]['aerosol'])
aerosol412 = pd.concat([aerosol412[x].apply(pd.Series) for x in list(aerosol412)], 1, keys = list(aerosol412))

plt.figure(figsize = (10, 5))
for m in model_names:
    plt.plot(aerosol412.index, aerosol412[m]['mean'], '-o', markersize='3')
    # plt.errorbar(aerosol412.index, aerosol412[m]['mean'], yerr = aerosol412[m]['std'], capsize = 4, fmt = '-o', markersize='3')
plt.ylim((0, 1))
plt.legend(model_names, ncol = 2)
plt.title('Sub-Actue Aerosol F1 score')
plt.show()
plt.close()

In [ ]:
#
vapor413 = pd.DataFrame(f1_dict[413]['vapour'])
vapor413 = pd.concat([vapor413[x].apply(pd.Series) for x in list(vapor413)], 1, keys = list(vapor413))

plt.figure(figsize = (10, 5))
for m in model_names:
    plt.plot(vapor413.index, vapor413[m]['mean'], '-o', markersize='3')
plt.ylim((0, 1))
plt.legend(model_names, ncol = 2)
plt.title('Sub-Chronic Vapor F1 score')
plt.show()
plt.close()

In [ ]:
#
aerosol413 = pd.DataFrame(f1_dict[413]['aerosol'])
aerosol413 = pd.concat([aerosol413[x].apply(pd.Series) for x in list(aerosol413)], 1, keys = list(aerosol413))

plt.figure(figsize = (10, 5))
for m in model_names:
    plt.plot(aerosol413.index, aerosol413[m]['mean'], '-o', markersize='3')
plt.ylim((0, 1))
plt.legend(model_names, ncol = 2)
plt.title('Sub-Chronic Aerosol F1 score')
plt.show()
plt.close()